Post simulation checks

In [ ]:
# Import modules
import numpy as np
import zarr
#import allel   
#idk why but my allel doesnt seem to load?? even though ive definitely installed it as scikit-allel
import scipy.cluster.hierarchy as sch
import scipy.spatial
import matplotlib
import matplotlib.pyplot as plt
import scipy.signal
from scipy.ndimage import gaussian_filter1d
from numpy.lib.stride_tricks import sliding_window_view
from tqdm import tqdm
import dask
from dask.delayed import delayed
from dask.base import compute
from itertools import combinations
import time
import seaborn as sns
import sys
import tskit

1. import in vcf/ tree file to examine population size


In [ ]:
# checking .tree outputs
import tskit
#tskit.load(file="C:/CHEYANNE STUFF/ICL Biology/Year 3/Final year project/burnin_no.19_early.trees")
tskit.load(file="C:/CHEYANNE STUFF/ICL Biology/Year 3/Final year project/burnin_no.19_early_10.trees")


# install pyslim !!! issue here involving msprime? pyslim cannot be imported/downloaded
#import pyslim

# read into pyslim and calculate nucleotide diversity and compare against expected nucleotide diversity (theta)
# check that it exceeds 10% and not more than 20%


2. Plot nucleotide diversity for all simulations.
- red flag if smaller than 10% or greater than 20%
- a. confidence intervals!